# 전기차 충전소 크롤링

In [262]:
# 모듈로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [263]:
# 연습
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('lang=ko_KR')

driver=webdriver.Chrome('C:/Users/USER/googlecrawing/chromedriver.exe',options=chrome_options)
driver.get('https://map.kakao.com/')

driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').send_keys(input_juso+Keys.ENTER)

time.sleep(3)

soup=BeautifulSoup(driver.page_source,'html.parser')
htmls=soup.select('.placelist .PlaceItem')

for html in htmls:
    print(html.select_one('a.link_name').text)
    print(html.find('p',attrs={'data-id':'address'}).text)

driver.close()

C:\Users\USER\AppData\Local\Temp\ipykernel_9684\1146195081.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('C:/Users/USER/googlecrawing/chromedriver.exe',options=chrome_options)


Tesla 대구 이시아폴리스 수퍼차저
대구 동구 봉무동 1544
Tesla 대구 수성 수퍼차저
대구 수성구 수성못6길 18 대구수성아트스퀘어
수태골 공영주차장 전기차충전소
대구 동구 용수동 907-1
대구신서혁신LH천년나무10단지 전기차충전소
대구 동구 메디밸리로 18
파계사 공영주차장 전기차충전소
대구 동구 파계로 686
Tesla 대구 수퍼차저
대구 북구 엑스코로 10
대덕맨션2차 전기차충전소
대구 남구 앞산순환로 655
동화사(동화사 집단시설지구 씨네80 극장 출구 주차장) 전기차충전소
대구 동구 팔공산로 1184
코스트코 대구혁신도시점 전기차충전소
대구 동구 첨단로 10
연경지구LH뉴웰시티아파트 전기차충전소
대구 동구 지묘동 1352-2
관음동한양수정아파트 전기차충전소
대구 북구 구암로 17
시지두레타운 입주자대표회의 전기차충전소
대구 수성구 천을로 131
현풍백년도깨비시장 공영주차장 전기차충전소
대구 달성군 현풍읍 현풍로6길 5
3호선 칠곡차량기지 전기차충전소
대구 북구 동호동 136-1
서대구역남측주차장 전기차충전소
대구 서구 와룡로 527


In [264]:
# 동적 웹 크롤링 함수(카카오맵)
## 동적 웹 사이트 옵션 설정
def kakaomap_crawing(input_search):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('lang=ko_KR') # 가짜 플러그인 탑재
    driver = webdriver.Chrome('C:/Users/USER/googlecrawing/chromedriver.exe',options=chrome_options)

    ## 크롤링할 사이트
    driver.get('https://map.kakao.com/')

    ## 사이트에 검색할 단어 입력 후 클릭
    driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]').send_keys(input_search+Keys.ENTER)

    ## 크롤링
    charge_name=[] # 충전소명 리스트
    address_name=[] # 충전소주소 리스트
    for i in range(1,35):
        time.sleep(1)
        if i==1:
            soup=BeautifulSoup(driver.page_source,'html.parser')
            names=soup.select('.placelist .PlaceItem')
            for name in names:
                charge_name.append(name.select_one('a.link_name').text)
                address_name.append(name.find('p',attrs={'data-id':'address'}).text)
        elif i==2:
            driver.find_element(By.XPATH,'//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)

            time.sleep(1)

            soup=BeautifulSoup(driver.page_source,'html.parser')
            names=soup.select('.placelist .PlaceItem')
            for name in names:
                charge_name.append(name.select_one('a.link_name').text)
                address_name.append(name.find('p',attrs={'data-id':'address'}).text)
        else:
            if i%5==1:
                driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
                
            driver.find_element(By.XPATH,f'//*[@id="info.search.page.no{5 if i%5==0 else i%5}"]').send_keys(Keys.ENTER)

            time.sleep(1)

            soup=BeautifulSoup(driver.page_source,'html.parser')
            names=soup.select('.placelist .PlaceItem')
            for name in names:
                charge_name.append(name.select_one('a.link_name').text)
                address_name.append(name.find('p',attrs={'data-id':'address'}).text)
    
    driver.close()

    return charge_name, address_name

In [265]:
charge_name, address_name=kakaomap_crawing('대구 전기차 충전소')

C:\Users\USER\AppData\Local\Temp\ipykernel_9684\3236740930.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/USER/googlecrawing/chromedriver.exe',options=chrome_options)


In [266]:
# 전기차 충전소명의 중복값 찾기
for x in range(len(charge_name)):
    if charge_name.count(charge_name[x])>=2:
        print(f"{x}인덱스의 {charge_name[x]} / {address_name[x]}")

13인덱스의 3호선 칠곡차량기지 전기차충전소 / 대구 북구 동호동 136-1
34인덱스의 수성소방서 전기차충전소 / 대구 수성구 청호로 87 건물 남측 1층 주차장
72인덱스의 수성소방서 전기차충전소 / 대구 수성구 범물동 1259
166인덱스의 동대구복합환승센터 전기차충전소 / 대구 동구 동부로 149 지하2층
199인덱스의 한국산업기술평가관리원 전기차충전소 / 대구 동구 신서동 1152
202인덱스의 한국산업기술평가관리원 전기차충전소 / 대구 수성구 청호로 87
304인덱스의 3호선 칠곡차량기지 전기차충전소 / 대구 북구 동호동 112-2
386인덱스의 동대구복합환승센터 전기차충전소 / 대구 동구 효신로 88


In [267]:
# 전기차 충전소 주소의 중복값 찾기
for x in range(len(address_name)):
    if address_name.count(address_name[x])>=2:
        print(f"{x}인덱스의 {address_name[x]} / {charge_name[x]}")

5인덱스의 대구 북구 엑스코로 10 / Tesla 대구 수퍼차저
10인덱스의 대구 북구 구암로 17 / 관음동한양수정아파트 전기차충전소
30인덱스의 대구 달서구 월배로32안길 61 / 귀빈맨션103동앞주차장 전기차충전소
106인덱스의 대구 북구 복현로 71 / 복현명문세가1단지103동 전기차충전소
160인덱스의 대구 수성구 두산동 499-4 / 대구환경공단 동부사업소 전기차충전소
205인덱스의 대구 달성군 옥포읍 본리로 102 / 옥포세광무지개마을 106동 전기차충전소
208인덱스의 대구 달서구 상화로 272 / LH 대구경북지역본부 전기차충전소
275인덱스의 대구 달서구 월배로32안길 61 / 귀빈맨션101동102동사이주차장 전기차충전소
311인덱스의 대구 달성군 유가읍 일연선사길 10 / 비슬산 유스호스텔 전기차충전소
320인덱스의 대구 서구 국채보상로50길 20 / 평리푸르지오1단지 전기차충전소
341인덱스의 대구 수성구 두산동 499-4 / 대구환경공단 전기차충전소
366인덱스의 대구 북구 구암로 17 / 한양수정상가관리단 전기차충전소
396인덱스의 대구 달성군 옥포읍 본리로 102 / 옥포세광무지개마을 105동 전기차충전소
413인덱스의 대구 달서구 상화로 272 / LH대구경북본부 전기차충전소
423인덱스의 대구 서구 국채보상로50길 20 / 평리푸르지오3단지 전기차충전소
449인덱스의 대구 달성군 유가읍 일연선사길 10 / 호텔아젤리아 전기차충전소
451인덱스의 대구 북구 엑스코로 10 / 북구유통단지 EXCO 전기차충전소
455인덱스의 대구 북구 복현로 71 / 복현명문세가1단지102동 전기차충전소


In [268]:
# 전기차 충전소 데이터 프레임 만들기
# elecar_df=pd.DataFrame({'name':charge_name,'address':juso_name})
elecar_df=pd.DataFrame([charge_name,address_name],index=['name','address']).T

In [270]:
# 전기차 충전소 데이터 프레임 csv파일 만들기
elecar_df.to_csv('C:/Users/USER/Desktop/해커톤/대구 전기차 충전소(건영).csv',index=False,encoding='cp949')

## 상언이형이 크롤링한 csv파일

In [ ]:
elecar=pd.read_csv('C:/Users/USER/Desktop/해커톤/대구 전기차 충전소.csv',encoding='cp949').iloc[:,1:]
elecar

,name,address
0,Tesla 대구 이시아폴리스 수퍼차저,대구 동구 봉무동 1544
1,Tesla 대구 수성 수퍼차저,대구 수성구 수성못6길 18 대구수성아트스퀘어
2,수태골 공영주차장 전기차충전소,대구 동구 용수동 907-1
3,대구신서혁신LH천년나무10단지 전기차충전소,대구 동구 메디밸리로 18
4,파계사 공영주차장 전기차충전소,대구 동구 파계로 686
...,...,...
490,달성습지생태학습관 전기차충전소,대구 달성군 화원읍 구라리 860
491,태전한일아파트 전기차충전소,대구 북구 칠곡중앙대로53길 32 102동 지하1층
492,블리스모텔 전기차충전소,대구 북구 칠곡중앙대로91길 5-7 옥내주차장 계단실 옆 모텔출입문 쪽
493,늘푸른교회 전기차충전소,대구 달서구 조암남로14길 13


In [ ]:
from geopy.geocoders import Nominatim

In [277]:
a=Nominatim(user_agent='South Korea',timeout=200)
b=a.geocode('대구 수성구 명덕로 455')
print(b[1])
print(b[1][0])
print(b[1][1])

(35.8542158, 128.6171196)
35.8542158
128.6171196


In [ ]:
elecar_latlng=elecar

In [ ]:
# lat_lng=Nominatim(user_agent='South Korea',timeout=200).geocode('대구 동구 용수동 907-1')
# print(lat_lng[0])

In [ ]:
# lat_lng=Nominatim(user_agent='South Korea',timeout=200).geocode(' '.join(elecar_latlng.loc[2,'address'].split()[0:4]))
# print(lat_lng[1][1])

In [ ]:
# for i in range(len(elecar_latlng)):
#     lat_lng=Nominatim(user_agent='South Korea',timeout=200).geocode(' '.join(elecar_latlng.loc[i,'address'].split()[0:4]))
#     print(i)
#     elecar_latlng.loc[i,'lat']=lat_lng[1][0]
#     elecar_latlng.loc[i,'lng']=lat_lng[1][1]

In [ ]:
# elecar.to_csv('C:/Users/USER/Desktop/해커톤/위도경도 대구 전기차 충전소.csv',index=False,encoding='cp949')